In [5]:
from trainer.glue_base import GlueAttentionApproxTrainer as Glue
import pickle

In [6]:
subsets = ["cola","mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli",]
kss = [0.1, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]

def get_score(score):
    if 'accuracy' in score:
        return score['accuracy']
    return score[list(score.keys())[0]]

results = {}
for subset in subsets:
    trainer = Glue(dataset=subset, factor=16, batch_size=-1, device=0)
    bert_score = get_score(trainer.eval_base_model())
    scores = {'bert':bert_score}
    for ks in kss:
        sparse_score = get_score(trainer.eval_sparse_model(ks=ks))
        scores[str(ks)] = sparse_score
    results[subset] = scores

with open('glue_benchmark.pkl', 'wb') as f:
    pickle.dump(results, f)

results

Trainer: cola


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-e8451e78068485b1.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-d3ce574ceba0d772.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-64c999469ec3ef94.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9d

metric score {'matthews_correlation': 0.5338774230813111}
avg occupy 0.34522852946633237


eval: 100%|██████████| 131/131 [00:07<00:00, 17.56it/s]


metric score {'matthews_correlation': -0.02674613601920812}
avg occupy 0.34522852946633237


...

eval: 100%|██████████| 9/9 [00:00<00:00, 15.23it/s]


metric score {'accuracy': 0.43661971830985913}
avg occupy 0.3578609244533317


eval: 100%|██████████| 9/9 [00:00<00:00, 15.23it/s]

metric score {'accuracy': 0.43661971830985913}
avg occupy 0.3578609244533317


{'cola': {'bert': 0.5338774230813111,
  '0.1': -0.02674613601920812,
  '0.25': 0.28027317122937173,
  '0.375': 0.42284522250559975,
  '0.5': 0.46464840528374507,
  '0.625': 0.47754454931020396,
  '0.75': 0.4917435371437847,
  '0.875': 0.49551502513336937},
 'mnli': {'bert': 0.8420784513499745,
  '0.1': 0.33662761079979625,
  '0.25': 0.3502801833927662,
  '0.375': 0.35476311767702495,
  '0.5': 0.3546612328069282,
  '0.625': 0.3556800815078961,
  '0.75': 0.35537442689760573,
  '0.875': 0.35578196637799286},
 'mrpc': {'bert': 0.8440579710144928,
  '0.1': 0.5634782608695652,
  '0.25': 0.6556521739130434,
  '0.375': 0.6666666666666666,
  '0.5': 0.6684057971014493,
  '0.625': 0.6753623188405797,
  '0.75': 0.6771014492753623,
  '0.875': 0.6765217391304348},
 'qnli': {'bert': 0.9154310818231741,
  '0.1': 0.47977301848801024,
  '0.25': 0.2624931356397584,
  '0.375': 0.20574775764232106,
  '0.5': 0.19897492220391727,
  '0.625': 0.1971444261394838,
  '0.75': 0.1975105253523705,
  '0.875': 0.19751